L1 pruning: torch.nn.utils.prune.l1_unstructured(module, name, amount, importance_scores=None)

Ln pruning: torch.nn.utils.prune.ln_structured(module, name, amount, n, dim, importance_scores=None)

In [1]:
import os
import copy
import time
import pandas as pd
from torchvision.io import read_image
from torchvision.datasets.folder import default_loader
from torchvision.datasets.utils import download_url
from torch.utils.data import Dataset
from torch import autograd
import torchvision.transforms as T
import matplotlib.pyplot as plt
from torchvision.models import vgg16
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.utils.prune as prune
from heapq import nsmallest
import torch.optim.lr_scheduler as lr_scheduler
import torch.optim as optim
import scipy.io
from os.path import join
from os import listdir
from torch.utils import data
from PIL import Image
import torchvision.datasets
from sklearn.model_selection import train_test_split

C:\Users\Victor\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!nvidia-smi

Fri Dec  9 16:27:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 522.06       Driver Version: 522.06       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:04:00.0  On |                  N/A |
| 44%   28C    P8    13W /  95W |   1271MiB /  2048MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla M40 24GB     TCC   | 00000000:2B:00.0 Off |           2679079386 |
| N/A   

In [3]:
BATCH_SIZE = 32

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
class Cub2011(Dataset):
    base_folder = 'CUB_200_2011/images'
    url = 'https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?download=1'
    filename = 'CUB_200_2011.tgz'
    tgz_md5 = '97eceeb196236b17998738112f37df78'

    def __init__(self, root, train=True, transform=None, loader=default_loader, download=True):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.loader = default_loader
        self.train = train

        if download:
            self._download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

    def _load_metadata(self):
        images = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'images.txt'), sep=' ',
                             names=['img_id', 'filepath'])
        image_class_labels = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'image_class_labels.txt'),
                                         sep=' ', names=['img_id', 'target'])
        train_test_split = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'train_test_split.txt'),
                                       sep=' ', names=['img_id', 'is_training_img'])

        data = images.merge(image_class_labels, on='img_id')
        self.data = data.merge(train_test_split, on='img_id')

        if self.train:
            self.data = self.data[self.data.is_training_img == 1]
        else:
            self.data = self.data[self.data.is_training_img == 0]

    def _check_integrity(self):
        try:
            self._load_metadata()
        except Exception:
            return False

        for index, row in self.data.iterrows():
            filepath = os.path.join(self.root, self.base_folder, row.filepath)
            if not os.path.isfile(filepath):
                print(filepath)
                return False
        return True

    def _download(self):
        import tarfile

        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        download_url(self.url, self.root, self.filename, self.tgz_md5)

        with tarfile.open(os.path.join(self.root, self.filename), "r:gz") as tar:
            tar.extractall(path=self.root)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        path = os.path.join(self.root, self.base_folder, sample.filepath)
        target = sample.target - 1 
        img = self.loader(path)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

In [6]:
transform = T.Compose([
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes,nclas, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_bal_acc = 0.0

    val_bal_acc = []
    val_acc = []
    val_loss = []

    train_bal_acc = []
    train_acc = []
    train_loss = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            CF = np.zeros((nclas,nclas)) # Confusion matrix

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device,non_blocking=True)
                labels = labels.to(device,non_blocking=True)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                for i in range(len(labels.data)):
                    CF[labels.data[i]][preds[i]] +=1

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            recalli = 0
            for i in range(nclas):
                TP = CF[i][i]
                FN = 0
                for j in range(nclas):
                    if i!=j:
                        FN+=CF[i][j]
                if (TP+FN) !=0:
                    recalli+= TP/(TP+FN)
            epoch_bal_acc = recalli/nclas

            if phase == 'val':
                val_bal_acc.append(epoch_bal_acc)
                val_acc.append(epoch_acc)
                val_loss.append(epoch_loss)
            else:
                train_bal_acc.append(epoch_bal_acc)
                train_acc.append(epoch_acc)
                train_loss.append(epoch_loss)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Balanced Acc: {epoch_bal_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_bal_acc > best_bal_acc:
                best_acc = epoch_acc
                best_bal_acc = epoch_bal_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    #print(f'Best val Acc: {best_acc:4f}')
    print(f'Best val Balanced Acc: {best_bal_acc:4f}')

    print('Validation:')
    print('Val_bal_acc:', val_bal_acc)
    print('Val_acc:', val_acc)
    print('Val_loss:', val_loss)

    print('Training:')
    print('Train_bal_acc:', train_bal_acc)
    print('Train_acc:', train_acc)
    print('Train_loss:', train_loss)

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

Local L1 unstructured pruning

In [9]:
#If finetuning == False -> Feature extraction
#Dataset = cub2011, stanford_dogs, caltech256
def imageNetPruningL1(pruningAmount, epochs, finetuning, transform, dataset):
  nclases = 0
  if(dataset == 'cub2011'):
    nclases = 200
    train_ds = Cub2011('./cub2011/train', train=True, transform = transform)
    val_ds = Cub2011('./cub2011/val', train=False, transform = transform)
  elif(dataset == 'pet'):
    nclases = 37
    train_ds = torchvision.datasets.OxfordIIITPet('./pet', split='trainval',  transform = transform,download=True)
    val_ds = torchvision.datasets.OxfordIIITPet('./pet', split='test', transform = transform,download=True)
  elif(dataset == 'dtd'):
    nclases = 47
    train_ds = torchvision.datasets.DTD('./dtd', split='train', transform = transform, download = True)
    val_ds = torchvision.datasets.DTD('./dtd', split='val', transform = transform, download = True)


  ds = {'train': DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
        'val': DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)}


  ds_sizes = {'train': len(train_ds),
        'val': len(val_ds)}

  model = vgg16(weights='IMAGENET1K_V1')
  model.classifier[6] = nn.Linear(4096, nclases) 

  if pruningAmount != 0:
    features = []
    classifier = []

    for n,p in model.named_parameters():
      if n.split('.')[0] == 'features' and n.split('.')[2] == 'weight':
        features.append(int(n.split('.')[1]))
      elif n.split('.')[0] == 'classifier' and n.split('.')[2] == 'weight':
        classifier.append(int(n.split('.')[1]))

    for x in features:
      prune.l1_unstructured(model.features[x], 'weight', amount=pruningAmount)
    for x in classifier:
      prune.l1_unstructured(model.classifier[x], 'weight', amount=pruningAmount)

  if(finetuning):
    optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
  else:
    optimizer_ft = optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)

  criterion = nn.CrossEntropyLoss()

  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

  model = model.to(device)

  return train_model(model, criterion, optimizer_ft, exp_lr_scheduler, ds, ds_sizes, nclases, num_epochs=epochs)

Local L2 unstructured pruning

In [10]:

def imageNetPruningL2(pruningAmount, epochs, finetuning, transform, dataset):
  nclases = 0
  if(dataset == 'cub2011'):
    nclases = 200
    train_ds = Cub2011('./cub2011/train', train=True, transform = transform)
    val_ds = Cub2011('./cub2011/val', train=False, transform = transform)
  elif(dataset == 'pet'):
    nclases = 37
    train_ds = torchvision.datasets.OxfordIIITPet('./pet', split='trainval',  transform = transform,download=True)
    val_ds = torchvision.datasets.OxfordIIITPet('./pet', split='test', transform = transform,download=True)
  elif(dataset == 'dtd'):
    nclases = 47
    train_ds = torchvision.datasets.DTD('./dtd', split='train', transform = transform, download = True)
    val_ds = torchvision.datasets.DTD('./dtd', split='val', transform = transform, download = True)


  ds = {'train': DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
        'val': DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)}


  ds_sizes = {'train': len(train_ds),
        'val': len(val_ds)}

  model = vgg16(weights='IMAGENET1K_V1')
  model.classifier[6] = nn.Linear(4096, nclases) 

  if pruningAmount != 0:
    features = []
    classifier = []

    for n,p in model.named_parameters():
      if n.split('.')[0] == 'features' and n.split('.')[2] == 'weight':
        features.append(int(n.split('.')[1]))
      elif n.split('.')[0] == 'classifier' and n.split('.')[2] == 'weight':
        classifier.append(int(n.split('.')[1]))

    for x in features:
      prune.l1_unstructured(model.features[x], 'weight', amount=pruningAmount)
    for x in classifier:
      prune.l1_unstructured(model.classifier[x], 'weight', amount=pruningAmount)

  if(finetuning):
    optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
  else:
    optimizer_ft = optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)

  criterion = nn.CrossEntropyLoss()

  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

  model = model.to(device)

  return train_model(model, criterion, optimizer_ft, exp_lr_scheduler, ds, ds_sizes, nclases, num_epochs=epochs)

In [ ]:
datasets = ['cub2011', 'pet','dtd']
print('L2')
for x in range(0, 7):
  pruningAmount = x/10
  print("Pruned "+ str(x*10)+'% ... local training with Fine Tuning')
  imageNetPruningL1(pruningAmount, 25, True, transform, datasets[0])
  #print("Pruned "+ str(x*10)+'% ... local training with Feature Extraction')
  #imageNetPruningL1(pruningAmount, 25, False, transform, datasets[1])




Pruned 50% ... local training with Fine Tuning


  0%|          | 0/1150585339 [00:00<?, ?it/s]

  0%|          | 0/1150585339 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

Epoch 0/24
----------
train Loss: 4.9887 Acc: 0.0355 Balanced Acc: 0.0355
val Loss: 3.7906 Acc: 0.1859 Balanced Acc: 0.1884

Epoch 1/24
----------
train Loss: 3.3942 Acc: 0.2224 Balanced Acc: 0.2224
val Loss: 2.5721 Acc: 0.3738 Balanced Acc: 0.3769

Epoch 2/24
----------
train Loss: 2.7027 Acc: 0.3293 Balanced Acc: 0.3293
val Loss: 2.3253 Acc: 0.4108 Balanced Acc: 0.4134

Epoch 3/24
----------
train Loss: 2.3427 Acc: 0.4054 Balanced Acc: 0.4054
val Loss: 1.9834 Acc: 0.4948 Balanced Acc: 0.4975

Epoch 4/24
----------
train Loss: 2.0657 Acc: 0.4640 Balanced Acc: 0.4639
val Loss: 1.9230 Acc: 0.5035 Balanced Acc: 0.5057

Epoch 5/24
----------
train Loss: 1.9484 Acc: 0.4947 Balanced Acc: 0.4946
val Loss: 1.8121 Acc: 0.5362 Balanced Acc: 0.5391

Epoch 6/24
----------
train Loss: 1.7808 Acc: 0.5269 Balanced Acc: 0.5268
val Loss: 1.7695 Acc: 0.5440 Balanced Acc: 0.5465

Epoch 7/24
----------
train Loss: 1.5602 Acc: 0.5919 Balanced Acc: 0.5919
val Loss: 1.6161 Acc: 0.5846 Balanced Acc: 0.5876



Global L1 Unstructured pruning

In [11]:
def imageNetPruningCUBL1GlobalUnstr(pruningAmount, epochs, finetuning, transform, dataset): 
  model = vgg16(weights='IMAGENET1K_V1')
  model.classifier[6] = nn.Linear(4096, 200) 
  parameters_to_prune = [
      (module, "weight") for module in filter(lambda m: type(m) == torch.nn.Conv2d or type(m) == torch.nn.Linear, model.modules())
  ]

  prune.global_unstructured(
      parameters_to_prune,
      pruning_method=prune.L1Unstructured,
      amount=pruningAmount,
  )

  for n,p in model.named_parameters():
    if n.split('.')[0] == 'features' and n.split('.')[2] == 'weight_orig':
      print(n, p.data.shape, p.data.numel(), p.data.numel() * pruningAmount)
    elif n.split('.')[0] == 'classifier' and n.split('.')[2] == 'weight_orig':
      print(n, p.data.shape, p.data.numel(), p.data.numel() * pruningAmount)

  if(dataset == 'cub2011'):
    nclases = 200
    train_ds = Cub2011('./cub2011/train', train=True, transform = transform)
    val_ds = Cub2011('./cub2011/val', train=False, transform = transform)
  elif(dataset == 'pet'):
    nclases = 37
    train_ds = torchvision.datasets.OxfordIIITPet('./pet', split='trainval',  transform = transform,download=True)
    val_ds = torchvision.datasets.OxfordIIITPet('./pet', split='test', transform = transform,download=True)
  elif(dataset == 'dtd'):
    nclases = 47
    train_ds = torchvision.datasets.DTD('./dtd', split='train', transform = transform, download = True)
    val_ds = torchvision.datasets.DTD('./dtd', split='val', transform = transform, download = True)

  if(finetuning):
    optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
  else:
    optimizer_ft = optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)


  ds = {'train': DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
        'val': DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)}


  ds_sizes = {'train': len(train_ds),
        'val': len(val_ds)}

  criterion = nn.CrossEntropyLoss()

  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

  model = model.to(device)

  return #train_model(model, criterion, optimizer_ft, exp_lr_scheduler, ds, ds_sizes, 200, num_epochs=epochs)
  



In [ ]:
imageNetPruningCUBL1GlobalUnstr(0.2, 25, True, transform, 'cub2011')
imageNetPruningCUBL1GlobalUnstr(0.1, 25, True, transform, 'pet')
imageNetPruningCUBL1GlobalUnstr(0.2, 25, True, transform, 'dtd')

features.0.bias
features.0.weight_orig
features.0.weight_orig torch.Size([64, 3, 3, 3]) 1728 345.6
features.2.bias
features.2.weight_orig
features.2.weight_orig torch.Size([64, 64, 3, 3]) 36864 7372.8
features.5.bias
features.5.weight_orig
features.5.weight_orig torch.Size([128, 64, 3, 3]) 73728 14745.6
features.7.bias
features.7.weight_orig
features.7.weight_orig torch.Size([128, 128, 3, 3]) 147456 29491.2
features.10.bias
features.10.weight_orig
features.10.weight_orig torch.Size([256, 128, 3, 3]) 294912 58982.4
features.12.bias
features.12.weight_orig
features.12.weight_orig torch.Size([256, 256, 3, 3]) 589824 117964.8
features.14.bias
features.14.weight_orig
features.14.weight_orig torch.Size([256, 256, 3, 3]) 589824 117964.8
features.17.bias
features.17.weight_orig
features.17.weight_orig torch.Size([512, 256, 3, 3]) 1179648 235929.6
features.19.bias
features.19.weight_orig
features.19.weight_orig torch.Size([512, 512, 3, 3]) 2359296 471859.2
features.21.bias
features.21.weight_or

  0%|          | 0/1150585339 [00:00<?, ?it/s]

  0%|          | 0/1150585339 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


features.0.bias
features.0.weight_orig
features.0.weight_orig torch.Size([64, 3, 3, 3]) 1728 172.8
features.2.bias
features.2.weight_orig
features.2.weight_orig torch.Size([64, 64, 3, 3]) 36864 3686.4
features.5.bias
features.5.weight_orig
features.5.weight_orig torch.Size([128, 64, 3, 3]) 73728 7372.8
features.7.bias
features.7.weight_orig
features.7.weight_orig torch.Size([128, 128, 3, 3]) 147456 14745.6
features.10.bias
features.10.weight_orig
features.10.weight_orig torch.Size([256, 128, 3, 3]) 294912 29491.2
features.12.bias
features.12.weight_orig
features.12.weight_orig torch.Size([256, 256, 3, 3]) 589824 58982.4
features.14.bias
features.14.weight_orig
features.14.weight_orig torch.Size([256, 256, 3, 3]) 589824 58982.4
features.17.bias
features.17.weight_orig
features.17.weight_orig torch.Size([512, 256, 3, 3]) 1179648 117964.8
features.19.bias
features.19.weight_orig
features.19.weight_orig torch.Size([512, 512, 3, 3]) 2359296 235929.6
features.21.bias
features.21.weight_orig


  0%|          | 0/791918971 [00:00<?, ?it/s]

Extracting pet/oxford-iiit-pet/images.tar.gz to pet/oxford-iiit-pet


  0%|          | 0/19173078 [00:00<?, ?it/s]

Extracting pet/oxford-iiit-pet/annotations.tar.gz to pet/oxford-iiit-pet


Local Structured pruning

In [12]:
def imageNetPruningLnStructured(pruningAmount, epochs, finetuning, transform, dataset, norm):
  nclases = 0
  if(dataset == 'cub2011'):
    nclases = 200
    train_ds = Cub2011('./cub2011/train', train=True, transform = transform)
    val_ds = Cub2011('./cub2011/val', train=False, transform = transform)
  elif(dataset == 'pet'):
    nclases = 37
    train_ds = torchvision.datasets.OxfordIIITPet('./pet', split='trainval',  transform = transform,download=True)
    val_ds = torchvision.datasets.OxfordIIITPet('./pet', split='test', transform = transform,download=True)
  elif(dataset == 'dtd'):
    nclases = 47
    train_ds = torchvision.datasets.DTD('./dtd', split='train', transform = transform, download = True)
    val_ds = torchvision.datasets.DTD('./dtd', split='val', transform = transform, download = True)


  ds = {'train': DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
        'val': DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)}


  ds_sizes = {'train': len(train_ds),
        'val': len(val_ds)}

  model = vgg16(weights='IMAGENET1K_V1')
  model.classifier[6] = nn.Linear(4096, nclases) 

  if pruningAmount != 0:
    features = []
    classifier = []
    for n,p in model.named_parameters():
      if n.split('.')[0] == 'features' and n.split('.')[2] == 'weight':
        features.append(int(n.split('.')[1]))
      elif n.split('.')[0] == 'classifier' and n.split('.')[2] == 'weight':
        classifier.append(int(n.split('.')[1]))
    for x in features:
      prune.ln_structured(model.features[x], 'weight', amount=pruningAmount, n = norm, dim=1)
    for x in classifier:
      prune.ln_structured(model.classifier[x], 'weight', amount=pruningAmount, n= norm, dim=1)

  if(finetuning):
    optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
  else:
    optimizer_ft = optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)

  criterion = nn.CrossEntropyLoss()

  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

  model = model.to(device)

  return train_model(model, criterion, optimizer_ft, exp_lr_scheduler, ds, ds_sizes, nclases, num_epochs=epochs)

In [ ]:
datasets = ['cub2011', 'pet','dtd']
for l in [1,2]:
  for dataset in datasets:
    print('Dataset: '+dataset)
    for x in range(3, 4):
        pruningAmount = x/10
        print("Pruned "+ str(x*10)+'% ... local training with Fine Tuning, L'+str(l))
        imageNetPruningLnStructured(pruningAmount, 25, True, transform, dataset, l)
        print("Pruned "+ str(x*10)+'% ... local training with Feature Extraction L'+str(l))
        imageNetPruningLnStructured(pruningAmount, 25, False, transform, dataset, l)

Dataset: cub2011
Pruned 30% ... local training with Fine Tuning, L1
Files already downloaded and verified
Files already downloaded and verified
Epoch 0/24
----------


Lottery Ticket Pruning

In [8]:
def lotteryTicketPruning(pruningAmount, iteration_epochs, iterations, transform, dataset, norm):

  s = 1-pruningAmount #Sparsity = 1 - %pruning

  amnt = s**(1/iterations) # Sparsity each iteration
  sparsity = 1

  init_parameters = {}

  if(dataset == 'cub2011'):
    nclases = 200
    train_ds = Cub2011('./cub2011/train', train=True, transform = transform)
    val_ds = Cub2011('./cub2011/val', train=False, transform = transform)
  elif(dataset == 'pet'):
    nclases = 37
    train_ds = torchvision.datasets.OxfordIIITPet('./pet', split='trainval',  transform = transform,download=True)
    val_ds = torchvision.datasets.OxfordIIITPet('./pet', split='test', transform = transform,download=True)
  elif(dataset == 'dtd'):
    nclases = 47
    train_ds = torchvision.datasets.DTD('./dtd', split='train', transform = transform, download = True)
    val_ds = torchvision.datasets.DTD('./dtd', split='val', transform = transform, download = True)


  ds = {'train': DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
        'val': DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)}


  ds_sizes = {'train': len(train_ds),
        'val': len(val_ds)}

  model = vgg16(weights='IMAGENET1K_V1')
  model.classifier[6] = nn.Linear(4096, nclases) 


  #Copy original parameters
  for n,p in model.named_parameters():
      init_parameters[n] = p.data.clone()

  for x in range(iterations):
    print('Current sparsity')
    print(sparsity)
    sparsity = sparsity*amnt
    

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    model = model.to(device)
    model = train_model(model, criterion, optimizer, exp_lr_scheduler, ds, ds_sizes, nclases, num_epochs=iteration_epochs)

    for i,module in enumerate(model.modules()):
      if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
        prune.ln_structured(module, 'weight', amount=1-amnt, n= norm, dim=1 )

    for n,p in model.named_parameters():
          n2 = n.replace('_orig','')
          if n2 in init_parameters:
            p.data = init_parameters[n2].clone()
    model = model.to(device)
    
    for i,module in enumerate(model.modules()):
      if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
        prune.remove(module, 'weight')
        #print(module.weight)

  

In [ ]:
for dataset in ['dtd','pet','cub2011' ]:
  for norm in [1,2]:
    for amount in [0.5]:
      print('Dataset: '+dataset)
      print('Norm L'+str(norm))
      print('Amount: '+str(amount))
      lotteryTicketPruning(amount, 15, 5, transform, dataset, norm)

Dataset: dtd
Norm L1
Amount: 0.5
Current sparsity
1
Epoch 0/14
----------
train Loss: 3.2279 Acc: 0.2032 Balanced Acc: 0.2032
val Loss: 2.1007 Acc: 0.4500 Balanced Acc: 0.4500

Epoch 1/14
----------
train Loss: 1.8135 Acc: 0.4941 Balanced Acc: 0.4941
val Loss: 1.8466 Acc: 0.4798 Balanced Acc: 0.4798

Epoch 2/14
----------
train Loss: 1.4164 Acc: 0.5941 Balanced Acc: 0.5941
val Loss: 1.6426 Acc: 0.5574 Balanced Acc: 0.5574

Epoch 3/14
----------
train Loss: 1.1810 Acc: 0.6590 Balanced Acc: 0.6590
val Loss: 1.6272 Acc: 0.5654 Balanced Acc: 0.5654

Epoch 4/14
----------
train Loss: 1.0150 Acc: 0.7037 Balanced Acc: 0.7037
val Loss: 1.6902 Acc: 0.5622 Balanced Acc: 0.5622

Epoch 5/14
----------
train Loss: 0.8693 Acc: 0.7271 Balanced Acc: 0.7271
val Loss: 1.6720 Acc: 0.5798 Balanced Acc: 0.5798

Epoch 6/14
----------
train Loss: 0.7903 Acc: 0.7638 Balanced Acc: 0.7638
val Loss: 1.6428 Acc: 0.5798 Balanced Acc: 0.5798

Epoch 7/14
----------
train Loss: 0.5497 Acc: 0.8324 Balanced Acc: 0.8324